In [ ]:
import os
import dask
from time import sleep
import utils.dirutils as dd
from dask.distributed import Client
import cupy as cp
import numpy as np
from dask_jobqueue import SLURMCluster
# import scipy as sp
# from numba import njit
import computation2_cluster as cc
import utils.dirutils as dd

In [3]:
(18**(3/2)-2**(3/2))*1/12

6.128258770283413

In [6]:
import numpy as np
13*np.sqrt(2)/3

6.128258770283413

In [ ]:
path = dd.base()
daskpath=os.path.join(path, "dask-worker-space-can-be-deleted")
try: 
    os.mkdir(daskpath) 
except OSError as error: 
    print(error)		 

# proc * threads = 18(cores)
# cluster = SLURMCluster(log_directory=daskpath) 
cluster = SLURMCluster(
    cores=12,
    queue='gpu',
    processes=3,
    log_directory=daskpath
)   

print(cluster.job_script())
cluster.scale(jobs=1) # 1 nodes
sleep(15)
client = Client(cluster)
client.amm.start()

In [ ]:
client

In [ ]:
P1 = np.arange(1,3)
P2 = np.arange(3,5)
P3 = np.arange(5,7)
fileformat = "L{}widerbeam.h5"
nj = 100000
stepj = nj//10

In [ ]:
def setup_gpu_worker(i):
    sleep(1.5)
    cp.cuda.Device(i).use()
    sleep(1.5)
with dask.annotate(resources={'gpu': 1}): # must included. proc--one-to-one--gpu
    setupGPUs = [ client.submit( setup_gpu_worker, i ) for i in range(3) ]

In [ ]:
with dask.annotate(resources={'gpu': 1}): # must included. proc--one-to-one--gpu
    future1 = client.submit(cc.computation, P1, nj, stepj, fileformat)
    future2 = client.submit(cc.computation, P2, nj, stepj, fileformat)
    future3 = client.submit(cc.computation, P3, nj, stepj, fileformat)
# client.close()
# cluster.close()

In [ ]:
client.cancel(future1)
client.cancel(future2)
client.cancel(future3)

In [ ]:
client.close()
cluster.close()

In [ ]:
client.restart()

In [ ]:
# cd ~/Data